In [ ]:
import os
from glob import glob
import cv2
import open3d as o3d
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 10)

In [ ]:
DATA_PATH = r'/media/joseph/Development/SFITC/kitti/data_scene_flow/training/'
RAW_DATA_PATH = r'2011_09_26/2011_09_26_drive_0057_sync'

left_image_paths = sorted(glob(os.path.join(RAW_DATA_PATH, 'image_02/data/0000000125.png')))
right_image_paths = sorted(glob(os.path.join(RAW_DATA_PATH, 'image_03/data/0000000125.png')))

# get LiDAR data
gt_paths = sorted(glob(os.path.join(DATA_PATH, 'disp_noc_0/000112_10.png')))

lidar_paths = sorted(glob(os.path.join(RAW_DATA_PATH, 'velodyne_points/data/0000000125.bin')))

print(f"Number of left images: {len(left_image_paths)}")
print(f"Number of right images: {len(right_image_paths)}")
print(f"Number of LiDAR point clouds: {len(lidar_paths)}")
print(f"Number of GT point clouds: {len(gt_paths)}")

In [ ]:
calib_paths = sorted(glob('/media/joseph/Development/SFITC/kitti/data_scene_flow_calib/training/calib_cam_to_cam/*.txt'))

with open(calib_paths[0],'r') as f:
    calib = f.readlines()

# get projection matrices
P_left = np.array([float(x) for x in calib[25].strip().split(' ')[1:]]).reshape((3,4))
P_right = np.array([float(x) for x in calib[33].strip().split(' ')[1:]]).reshape((3,4))

# get rectified rotation matrices
R_left_rect = np.array([float(x) for x in calib[24].strip().split(' ')[1:]]).reshape((3, 3,))
R_right_rect = np.array([float(x) for x in calib[32].strip().split(' ')[1:]]).reshape((3, 3,))

R_left_rect = np.insert(R_left_rect, 3, values=[0,0,0], axis=0)
R_left_rect = np.insert(R_left_rect, 3, values=[0,0,0,1], axis=1)

In [ ]:
def decompose_projection_matrix(P):    
    K, R, T, _, _, _, _ = cv2.decomposeProjectionMatrix(P)
    T = T/T[3]

    return K, R, T

In [ ]:
K_left, R_left, T_left = decompose_projection_matrix(P_left)
K_right, R_right, T_right = decompose_projection_matrix(P_right)

In [ ]:
print(T_left)
print(R_left)

In [ ]:
calib_paths = sorted(glob('/media/joseph/Development/SFITC/kitti/data_scene_flow_calib/training/calib_velo_to_cam/*.txt'))

with open(calib_paths[0], 'r') as f:
    calib = f.readlines()

R_cam_velo = np.array([float(x) for x in calib[1].strip().split(' ')[1:]]).reshape((3, 3))
t_cam_velo = np.array([float(x) for x in calib[2].strip().split(' ')[1:]])[:, None]

T_cam_velo = np.vstack((np.hstack((R_cam_velo, t_cam_velo)),
                        np.array([0, 0, 0, 1])))

T_cam_velo

In [ ]:
index = 0

left_image = cv2.cvtColor(cv2.imread(left_image_paths[index]), cv2.COLOR_BGR2RGB)
right_image = cv2.cvtColor(cv2.imread(right_image_paths[index]), cv2.COLOR_BGR2RGB)

In [ ]:
print(f"The shape of an image is: {left_image.shape}")

In [ ]:
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(25, 15))
ax1.imshow(left_image)
ax1.set_title('Left Image', size=22)
ax2.imshow(right_image)
ax2.set_title('Right Image', size=22);

In [ ]:
def compute_sgbm_disparity(left_image, right_image, num_disparities=5*16, 
                           block_size=11, window_size=5, display=False):
    """ Computes the disparity of an image pair using the SGBM algoithm.
        Inputs: 
            image_left/_right - (MxN) grayscale input images
            see opencv documentation for "StereoBM_create"
        Outputs:
            disparity (MxN) computed disparity map for the input images
        
        NOTE: image_left must be the left image (same for the right) or 
              unexpected results will occur due to 
    """
    # P1 and P2 control disparity smoothness (recommended values below)
    P1 = 8 * 3 * window_size**2
    P2 = 32 * 3 * window_size**2
    sgbm_obj = cv2.StereoSGBM_create(0, num_disparities, block_size, 
        P1, P2, mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY)
        
    # compute disparity
    disparity = sgbm_obj.compute(left_image, right_image).astype(np.float32)/16.0

    # display is desired
    if display:
      plt.figure(figsize = (40,20))
      plt.imshow(disparity, cmap='cividis')
      plt.title('Disparity Map', size=25)
      plt.show();

    return disparity

def calc_depth_map(disp_left, K_left, T_left, T_right):
    ''' Computes Depth map from Intrinsic Camera Matrix and Translations vectors.
        For KITTI, the depth is in meters.
        '''
    # Get the focal length from the K matrix
    f = K_left[0, 0]
    
    # Get the distance between the cameras from the t matrices (baseline)
    b = np.abs(T_left[0] - T_right[0])[0]
    
    # Replace all instances of 0 and -1 disparity with a small minimum value (to avoid div by 0 or negatives)
    disp_left[disp_left <= 0] = 1e-5
    
    # Calculate the depths 
    depth_map = f*b / disp_left 

    return depth_map

In [ ]:
left_image_gray = cv2.cvtColor(left_image, cv2.COLOR_RGB2GRAY)
right_image_gray = cv2.cvtColor(right_image, cv2.COLOR_RGB2GRAY)

In [ ]:
from IPython.display import Image
from ipywidgets import interact, interactive, fixed

disparity = interactive(compute_sgbm_disparity, 
                        left_image=fixed(left_image_gray), 
                        right_image=fixed(right_image_gray), 
                        num_disparities=(0,512,16), 
                        block_size=(1,19,2), 
                        window_size=(1,13,2),
                        display=fixed(True))
display(disparity)

In [ ]:
num_disparities = disparity.kwargs['num_disparities']
block_size = disparity.kwargs['block_size']
window_size = disparity.kwargs['window_size']

In [ ]:
disparity = compute_sgbm_disparity(left_image_gray, 
                                   right_image_gray, 
                                   num_disparities, 
                                   block_size, 
                                   window_size, 
                                   display=False)

In [ ]:
plt.imshow(disparity)

In [ ]:
gt = gt_paths[index]

gt_disp = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)
# print(gt, type(gt_depth_map))

gt_disp = gt_disp.astype(np.float32)
# gt_disp[gt_disp == 0] = 1e-05

gt_depth_map = calc_depth_map(gt_disp, K_left, T_left, T_right)
gt_depth_map[gt_depth_map > 38000000.0] = np.NAN
# gt_depth_map[gt_depth_map == 0] = np.NAN
# gt_depth_map[gt_depth_map == np.Infinity] = np.NAN

plt.imshow(gt_depth_map);
gt_depth_map[50,600]

In [ ]:
def get_velo2cam(lidar_bin):
    ''' Converts the LiDAR point cloud to camera (u, v, z) image coordinates, 
        where z is in meters
        '''
    # read in LiDAR data
    scan_data = np.fromfile(lidar_bin, dtype=np.float32).reshape((-1,4))

    # convert to homogeneous coordinate system
    velo_points = scan_data[:, 0:3] # (x, y, z) --> (front, left, up)
    velo_points = np.insert(velo_points, 3, 1, axis=1).T # homogeneous LiDAR points

    # delete negative liDAR points
    velo_points = np.delete(velo_points, np.where(velo_points[3, :] < 0), axis=1) 

    # possibly use RANSAC to remove the ground plane for better viewing?

    # convert to camera coordinates
    velo_camera = P_left @ R_left_rect @ T_cam_velo @ velo_points

    # delete negative camera points ??
    velo_camera  = np.delete(velo_camera , np.where(velo_camera [2,:] < 0)[0], axis=1) 

    # get camera coordinates u,v,z
    velo_camera[:2] /= velo_camera[2, :]

    return velo_camera

def project_velo2cam(lidar_bin, image):
    ''' Projects LiDAR point cloud onto the image coordinate frame '''

    # get camera (u, v, z) coordinates
    velo_camera = get_velo2cam(lidar_bin)

    (u, v, z) = velo_camera

    # remove outliers (points outside of the image frame)
    img_h, img_w, _ = image.shape
    u_out = np.logical_or(u < 0, u > img_w)
    v_out = np.logical_or(v < 0, v > img_h)
    outlier = np.logical_or(u_out, v_out)
    velo_camera = np.delete(velo_camera, np.where(outlier), axis=1)
    
    return velo_camera

In [ ]:
lidar_bin = lidar_paths[index]
(u, v, z) = project_velo2cam(lidar_bin, left_image)

In [ ]:
def uvz_to_depth_map(uvz, image_shape):
    depth_map = np.zeros(image_shape, dtype=np.float32)
    
    u, v, z = uvz
    u, v = u.astype(np.int32), v.astype(np.int32)
    
    valid_indices = (u >= 0) & (u < image_shape[1]) & (v >= 0) & (v < image_shape[0])
    u, v, z = u[valid_indices], v[valid_indices], z[valid_indices]
    
    depth_map[v, u] = z
    
    return depth_map
    
lidar_depth_map = uvz_to_depth_map((u, v, z), stereo_depth_map.shape)

# TODO: Why not working?
lidar_depth_map[lidar_depth_map == 0] = np.NAN

plt.imshow(lidar_depth_map, cmap='rainbow_r');

In [ ]:
stereo_depth_map = calc_depth_map(disparity, K_left, T_left, T_right)
stereo_depth_map[stereo_depth_map > 38000000.0] = np.NAN

plt.imshow(stereo_depth_map, cmap='rainbow_r'); # or 'cividis_r' or 'cividis'

## Overlapping

In [ ]:
lidar_depth_map.shape

In [ ]:
# TODO: Make this dynamic

plt.imshow(lidar_depth_map[150:,:])

In [ ]:
plt.imshow(stereo_depth_map[:,80:])

## Point Cloud

In [ ]:
def depth_map_to_point_cloud(depth_map, flip=True):
    # Convert the depth map to Open3D depth image
    depth_image_o3d = o3d.geometry.Image(depth_map.astype(np.float32))

    # Create an Intrinsics object using camera parameters
    height, width = depth_map.shape
    fx, fy = K_left[0, 0], K_left[1, 1]
    cx, cy = K_left[0, 2], K_left[1, 2]
    intrinsic = o3d.camera.PinholeCameraIntrinsic(
        width, height, fx, fy, cx, cy)

    # Create a point cloud from the depth image
    point_cloud = o3d.geometry.PointCloud.create_from_depth_image(
        depth_image_o3d, 
        intrinsic,
        # depth_scale=1,
        # depth_trunc=0.00070
    )

    if flip:
        point_cloud.transform([
            [1, 0, 0, 0],
            [0, -1, 0, 0],
            [0, 0, -1, 0],
            [0, 0, 0, 1]
        ])

    return point_cloud


def rgbd_to_point_cloud(color, depth_map, flip=True):
    # Convert the depth map to Open3D depth image
    depth_image_o3d = o3d.geometry.Image(depth_map.astype(np.float32))

    # Convert the depth map to Open3D depth image
    color_image_o3d = o3d.geometry.Image(color)

    # Create RGBD image
    rgbd_image_o3d = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color_image_o3d, 
        depth_image_o3d,
        depth_scale=1,
        depth_trunc=70
        )

    # Create an Intrinsics object using camera parameters
    height, width = depth_map.shape
    fx, fy = K_left[0, 0], K_left[1, 1]
    cx, cy = K_left[0, 2], K_left[1, 2]
    intrinsic = o3d.camera.PinholeCameraIntrinsic(
        width, height, fx, fy, cx, cy)

    # Create a point cloud from the depth image
    point_cloud = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image_o3d, intrinsic)

    if flip:
        point_cloud.transform([
            [1, 0, 0, 0],
            [0, -1, 0, 0],
            [0, 0, -1, 0],
            [0, 0, 0, 1]
        ])

    return point_cloud

# # Visualize the point cloud
# o3d.visualization.draw_geometries([
#     # depth_map_to_point_cloud(stereo_depth_map),
#     depth_map_to_point_cloud(lidar_depth_map),
#     rgbd_to_point_cloud(left_image, stereo_depth_map),
# ])

## PG Implementation

In [ ]:
import cupy as cp

# vlp_depth = cp.array(lidar_depth_map)
# zed_depth = cp.array(stereo_depth_map)
# rgb = cp.array(left_image)
gt_depth = cp.array(gt_depth_map[150:,80:])
vlp_depth = cp.array(lidar_depth_map[150:,80:])
zed_depth = cp.array(stereo_depth_map[150:,80:])
rgb = cp.array(left_image[150:,80:])

def fillB_withA(A, B):
    # Combine A and B, prioritizing non-NaN values in A
    combined = cp.where(cp.isnan(A), B, A)

    # Calculate the column-wise means of the combined matrix, ignoring NaNs
    col_means = cp.nanmean(combined, axis=0)

    # Replace NaN values in the combined matrix with the column-wise means
    combined = cp.where(cp.isnan(combined), col_means, combined)
    
    return combined

print(cp.count_nonzero(cp.isnan(vlp_depth)))
print(cp.count_nonzero(cp.isnan(zed_depth)))

mask = cp.isnan(zed_depth)

zed_depth = cp.array(cv2.inpaint(zed_depth.get(), mask.get().astype(np.uint8), 3, cv2.INPAINT_TELEA))

# zed_depth[cp.isnan(zed_depth)] = vlp_depth.mean()
# zed_depth[cp.isnan(zed_depth)] = vlp_depth.max()
# zed_depth[cp.isnan(zed_depth)] = vlp_depth.min()
# zed_depth[zed_depth > 2000] = vlp_mean

zed_depth = fillB_withA(vlp_depth, zed_depth)

print(cp.count_nonzero(cp.isnan(zed_depth)))

plt.imshow(np.log(zed_depth.get()), cmap='rainbow_r');

In [ ]:
def lpf(img, ncutoff):
    # Apply 2D FFT to the image
    f = cp.fft.fft2(img)

    # Shift the zero frequency component to the center of the spectrum
    fshift = cp.fft.fftshift(f)

    # Create a circular mask of the same size as the spectrum
    rows, cols = img.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols), np.uint8)
    cutoff = int(min(crow, ccol)*ncutoff)
    cv2.circle(mask, (ccol, crow), cutoff, 1, -1)
    # cv2.ellipse(mask, (ccol, crow), (1, 2) * cutoff, 0, 0, 360,  1, -1)

    mask = cp.asarray(mask)

    # Apply the mask to the shifted spectrum
    fshift_filtered = fshift * mask

    # Shift the zero frequency component back to the corner of the spectrum
    f_filtered = cp.fft.ifftshift(fshift_filtered)

    # Apply the inverse 2D FFT to the filtered spectrum
    img_filtered = cp.fft.ifft2(f_filtered)
    img_filtered = cp.real(img_filtered)

    return img_filtered


def pg(zed_depth, vlp_depth, ncutoff, threshold=100):
    mask = vlp_depth > 0
    filtered = zed_depth

    while threshold > 0:
        filtered[mask] = vlp_depth[mask]
        filtered = lpf(filtered, ncutoff)

        threshold -= 1
        # ncutoff = ncutoff / 10

    return filtered


pg_depth = pg(
    cp.array(zed_depth.copy()),
    cp.array(vlp_depth.copy()),
    # cp.log(cp.array(zed_depth.copy())),
    # cp.log(cp.array(vlp_depth.copy())),
    ncutoff=10,
    threshold=1000
)

# pg_depth = cp.exp(pg_depth)
pg_depth[mask] = cp.NAN
masked_zed_depth = zed_depth
masked_zed_depth[mask] = cp.NAN

# plt.imshow(pg_depth.get(), cmap='rainbow_r')
# plt.imshow(np.log(pg_depth.get()), cmap='rainbow_r')
# plt.imshow(np.exp(pg_depth.get()), cmap='rainbow_r')

plt.subplot(3, 1, 1)
plt.imshow(np.log(masked_zed_depth.get()), cmap='rainbow_r')
plt.title('Zed')

plt.subplot(3, 1, 2)
plt.imshow(np.log(pg_depth.get()), cmap='rainbow_r')
plt.title('PG')

plt.subplot(3, 1, 3)
plt.imshow(np.log(pg_depth.get() - zed_depth.get()), cmap='rainbow_r')
plt.title('Diff')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize the point cloud
# o3d.visualization.draw_geometries([
#     # depth_map_to_point_cloud(zed_depth.get()),
#     rgbd_to_point_cloud(rgb.get(), masked_zed_depth.get()),
#     # depth_map_to_point_cloud(vlp_depth.get()),
#     # depth_map_to_point_cloud(gt_depth.get()),
#     # rgbd_to_point_cloud(rgb.get(), pg_depth.get()),
#     # depth_map_to_point_cloud(pg_depth.get()),
# ])


## Benchmark


In [ ]:
def calculate_mae(predicted, ground_truth):
    """
    Calculate the Mean Absolute Error (MAE) between predicted and ground truth images,
    ignoring NaN values.

    Parameters:
    predicted (numpy.ndarray): Predicted depth image.
    ground_truth (numpy.ndarray): Ground truth depth image.

    Returns:
    float: The MAE value.
    """
    # Ensure the images have the same shape
    if predicted.shape != ground_truth.shape:
        raise ValueError("Predicted and ground truth images must have the same shape.")

    # Create a mask for valid (non-NaN) pixels
    valid_mask = ~np.isnan(predicted) & ~np.isnan(ground_truth)

    # Calculate the absolute differences only for valid pixels
    diffs = np.abs(predicted[valid_mask] - ground_truth[valid_mask])

    # Calculate the mean of these differences
    mae = np.mean(diffs)
    return mae

print(calculate_mae(gt_depth.get(), gt_depth.get()))
print(calculate_mae(pg_depth.get(), gt_depth.get()))
print(calculate_mae(masked_zed_depth.get(), gt_depth.get()))
print(calculate_mae(vlp_depth.get(), gt_depth.get()))

In [ ]:
def calculate_custom_mae(predicted, ground_truth):
    """
    Calculate a custom Mean Absolute Error (MAE) between predicted and ground truth images.
    NaN values in the ground truth are ignored. NaN values in the predicted image, where
    the ground truth is not NaN, are penalized.

    Parameters:
    predicted (numpy.ndarray): Predicted depth image.
    ground_truth (numpy.ndarray): Ground truth depth image.

    Returns:
    float: The custom MAE value.
    """
    # Ensure the images have the same shape
    if predicted.shape != ground_truth.shape:
        raise ValueError("Predicted and ground truth images must have the same shape.")

    # Mask for valid pixels in ground truth
    valid_gt_mask = ~np.isnan(ground_truth)

    # Mask for pixels where predicted is NaN but ground truth is not
    bad_pred_mask = np.isnan(predicted) & valid_gt_mask

    # Assign a large error value for bad predictions
    bad_pred_penalty = 1000  # or some other large number representing a high error

    # Calculate absolute differences where both are valid and add penalty for bad predictions
    diffs = np.abs(np.where(bad_pred_mask, bad_pred_penalty, predicted) - ground_truth)

    # Calculate the mean over the ground truth valid mask
    mae = np.mean(diffs[valid_gt_mask])
    return mae

print(calculate_custom_mae(gt_depth.get(), gt_depth.get()))
print(calculate_custom_mae(pg_depth.get(), gt_depth.get()))
print(calculate_custom_mae(masked_zed_depth.get(), gt_depth.get()))
print(calculate_custom_mae(vlp_depth.get(), gt_depth.get()))

In [ ]:
print(np.nan_to_num(pg_depth.get() - masked_zed_depth.get(), 0).max())
print(np.nan_to_num(gt_depth.get() - masked_zed_depth.get(), 0).max())
print(np.nan_to_num(vlp_depth.get() - masked_zed_depth.get(), 0).max())

In [ ]:
def calculate_rmse(predicted, ground_truth):
    """
    Calculate the Root Mean Square Error (RMSE) between predicted and ground truth images,
    taking into account NaN values.

    Parameters:
    predicted (numpy.ndarray): Predicted depth image.
    ground_truth (numpy.ndarray): Ground truth depth image.

    Returns:
    float: The RMSE value.
    """
    # Ensure the images have the same shape
    if predicted.shape != ground_truth.shape:
        raise ValueError("Predicted and ground truth images must have the same shape.")

    # Mask for valid pixels in ground truth
    valid_gt_mask = ~np.isnan(ground_truth)

    # Mask for pixels where predicted is NaN but ground truth is not
    bad_pred_mask = np.isnan(predicted) & valid_gt_mask

    # Assign a large error value for bad predictions
    bad_pred_penalty = 1000  # or some other large number representing a high error

    # Calculate squared differences where both are valid and add penalty for bad predictions
    squared_diffs = (np.where(bad_pred_mask, bad_pred_penalty, predicted) - ground_truth) ** 2

    # Calculate the mean of the squared differences over the ground truth valid mask and then take the square root
    rmse = np.sqrt(np.mean(squared_diffs[valid_gt_mask]))
    return rmse


print(calculate_rmse(gt_depth.get(), gt_depth.get()))
print(calculate_rmse(pg_depth.get(), gt_depth.get()))
print(calculate_rmse(masked_zed_depth.get(), gt_depth.get()))
print(calculate_rmse(vlp_depth.get(), gt_depth.get()))